In [1]:
# Import essential libraries
import cv2
import numpy as np

def test_orb(orb, img):
    from matplotlib import pyplot as plt
    # find the keypoints with ORB
    kp = orb.detect(img,None)
    # compute the descriptors with ORB
    kp, des = orb.compute(img, kp)
    # draw only keypoints location,not size and orientation
    img2 = cv2.drawKeypoints(img, kp, None, color=(0,255,0), flags=0)
    return img2

def detect_image_T(orb, imgF, imgT, kpT, desT):
    
    # Initialize Brute-Force matcher
    bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)

    # Detect keypoints and compute descriptors for the resized image
    kpF, desF = orb.detectAndCompute(imgF, None)
    
    # Match descriptors between template and resized image
    matches = bf.match(desF, desT)

    # Sort them in the order of their distance.
    matches = sorted(matches, key = lambda x:x.distance)
    
    # Draw first 10 matches.
    imgFD = cv2.drawMatches(imgF,kpF,imgT,kpT,matches[:10],None,flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS)

    return imgFD

url = "http://192.168.50.15:8080/video"
cap = cv2.VideoCapture(url)

img1 = cv2.imread('object1.png', 0)

# Initialize ORB detectorasd
orb = cv2.ORB_create()

# Detect keypoints and compute descriptors for the template image
kp1, des1 = orb.detectAndCompute(img1, None)


while(True):
    ret, imgF = cap.read()
    
    #matched_image = test_orb(orb, main_image)
    imgM = detect_image_T(orb, imgF, img1, kp1, des1)
    
    cv2.imshow('frame', cv2.resize(imgM, (540, 960)) )
    if cv2.waitKey(1) & 0xFF == ord('q'):
        cv2.destroyAllWindows()
        break


KeyboardInterrupt: 

In [ ]:
import numpy as np
import cv2 as cv
import matplotlib.pyplot as plt

url = "http://192.168.50.15:8080/video"
cap = cv2.VideoCapture(url)

img2 = cv2.imread('object1.png',cv2.IMREAD_GRAYSCALE) # trainImage


while(True):
    ret, img1 = cap.read()
    
    # Initiate SIFT detector
    sift = cv2.SIFT_create()
    # find the keypoints and descriptors with SIFT
    kp1, des1 = sift.detectAndCompute(img1,None)
    kp2, des2 = sift.detectAndCompute(img2,None)
    # BFMatcher with default params
    bf = cv2.BFMatcher()
    matches = bf.knnMatch(des1,des2,k=2)
    # Apply ratio test
    good = []
    for m,n in matches:
        if m.distance < 0.75*n.distance:
            good.append([m])
            # cv2.drawMatchesKnn expects list of lists as matches.
            img3 = cv2.drawMatchesKnn(img1,kp1,img2,kp2,good,None,flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS)

    
    cv2.imshow('frame', cv2.resize(img3, (540, 960)) )
    if cv2.waitKey(1) & 0xFF == ord('q'):
        cv2.destroyAllWindows()
        break

In [45]:
%reset -f
# Import essential libraries
import cv2
import numpy as np

MIN_MATCH_COUNT = 30

def compute_sift(img, kp, des, imgT, kpT, desT):
    found = 0
    matchesMask = None
    
    flann = cv2.FlannBasedMatcher(index_params, search_params)
    matches = flann.knnMatch(desT,des,k=2)
    
    # store all the good matches as per Lowe's ratio test.
    good = []
    for m,n in matches:
        if m.distance < 0.7*n.distance:
            good.append(m)
    
    # If enough matches are found, we extract the locations of matched keypoints in both the images.
    if len(good)>MIN_MATCH_COUNT:
        found = 1
        src_pts = np.float32([ kpT[m.queryIdx].pt for m in good ]).reshape(-1,1,2)
        dst_pts = np.float32([ kp[m.trainIdx].pt for m in good ]).reshape(-1,1,2)
        M, mask = cv2.findHomography(src_pts, dst_pts, cv2.RANSAC,5.0)
        matchesMask = mask.ravel().tolist()
        h,w = imgT.shape
        pts = np.float32([ [0,0],[0,h-1],[w-1,h-1],[w-1,0] ]).reshape(-1,1,2)
        dst = cv2.perspectiveTransform(pts,M)
        img = cv2.polylines(img,[np.int32(dst)],True,255,3, cv2.LINE_AA)
    #else:
    #    print( "Not enough matches are found - {}/{}".format(len(good), MIN_MATCH_COUNT), end='\r')
    
    # Finally we draw our inliers (if successfully found the object) 
    # or matching keypoints (if failed).
    draw_params = dict( matchColor = (0,255,0), # draw matches in green color
                        singlePointColor = None,
                        matchesMask = matchesMask, # draw only inliers
                        flags = 2
                    )
    
    return found, cv2.drawMatches(imgT, kpT, img, kp, good, None, **draw_params)



url = "http://192.168.50.15:8080/video"
cap = cv2.VideoCapture(url)

img_paths = ['object1.png','object2.png','object3.png']

# Initiate SIFT detector
sift = cv2.SIFT_create()

imgs_data = []
for path in img_paths:
    
    imgT = cv2.imread(path, 0)
    
    # Detect keypoints and compute descriptors for the template image
    kpT, desT = sift.detectAndCompute(imgT, None)
    
    imgs_data.append([path, imgT, kpT, desT])
    


# SIFT parameters
FLANN_INDEX_KDTREE = 1
index_params = dict(algorithm = FLANN_INDEX_KDTREE, trees = 5)
search_params = dict(checks = 50)

matches_found = {p:0 for p in img_paths}
on = True

while(on):
    ret, img = cap.read()
    
    # find the keypoints and descriptors with SIFT for every frame
    kp, des = sift.detectAndCompute(img, None)
    
    if len(kp) > 30:
        
        for (path, imgT, kpT, desT) in imgs_data:
            
            found, imgTM = compute_sift(img, kp, des, imgT, kpT, desT)
            matches_found[path] = found
            
            
            cv2.imshow('frame-%s'%path, cv2.resize(imgTM, (540, 960)) )
            if cv2.waitKey(1) & 0xFF == ord('q'):
                on = False
    print(matches_found, end='\r')
cv2.destroyAllWindows()